In [2]:
# evaluate an IMT system on a ranked version of the IMT F1 metric
import codecs

import numpy as np

from nn_imt.stream import imt_f1, map_pair_to_imt_triples

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 4007)


In [3]:
# WORKING: add a function to compute ranked NDCG -- add a demo evaluation using an n-best list


$ DCG_{r} = \sum_{i=1}^{r} \frac{2^{rel_{i}} - 1}{log_{2}(i+1)} $


In [29]:
(2**1 - 1) / np.log2(2)

1.0

In [45]:
def dcg(scores):
    num_scores = len(scores)
    assert num_scores > 0, 'you must pass a 1d iterable containing at least one score'
    
    scaled_scores = []
    for s,i in zip(scores, range(1, num_scores+1)):
        scaled_score = ((2**s) - 1) / np.log2(i+1)
        scaled_scores.append(scaled_score)
    return np.sum(scaled_scores)

# now add a function to compute idcg
In order to make this score comparable across different $(s_{i}, t_{i})$ queries, 
we normalize $ DCG_{r} $ by the `ideal' ranking at $ r $: ($ IDCG_{r} $), by sorting all predictions 
by their score from equation~\ref{eq:imt-f1}, and obtain the normalized score:

$ \mathbf{NDCG_{r}} = \frac{DCG_{r}}{IDCG_{r}} $


In [46]:
def ndcg(scores):
    ideal_dcg = dcg(sorted(scores, reverse=True))
    normed_cdg = dcg(scores) / ideal_dcg
    return normed_cdg

In [47]:
sorted([0.7,0.4,0.2], reverse=True)

[0.7, 0.4, 0.2]

In [48]:
ndcg([0.7,0.4,0.2])

1.0

In [49]:
ndcg([0.4,0.7,0.2,])

0.87498873088320561

In [51]:
import itertools
l = [0.7,0.4,0.2]
tl = itertools.permutations(l)

for t in tl:
    print(t)
    print(ndcg(t))
    
l = [0.7,0.4]
tl = itertools.permutations(l)

for t in tl:
    print(t)
    print(ndcg(t))

(0.7, 0.4, 0.2)
1.0
(0.7, 0.2, 0.4)
0.975163222679
(0.4, 0.7, 0.2)
0.874988730883
(0.4, 0.2, 0.7)
0.80580350007
(0.2, 0.7, 0.4)
0.78014101047
(0.2, 0.4, 0.7)
0.735792556977
(0.7, 0.4)
1.0
(0.4, 0.7)
0.863737578668


In [5]:
np.log(0)

-inf